In [1]:
import sys, os
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import transforms
import nibabel as nib
import nrrd
import random
from omegaconf import OmegaConf
from typing import Iterable, Dict, Callable, Tuple, Union
from scipy.ndimage import binary_erosion, binary_dilation
import random


sys.path.append('..')
from utils import *
from model import get_model

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
class UserModel:
    
    def __init__(
        self, 
        ground_truth: Tensor, 
        cfg: dict, 
        # brush_sizes=torch.arange(2,7)
    ):
        super().__init__()
        
        # globals
        self.gt = ground_truth.float() # nd array or float tensor?
        # if 'brush' in cfg.keys():
        self.brush_sizes = cfg['brush_sizes']
        # else:
        #     self.brush_sizes = [1]
        
        if cfg['slice_selection'] == 'mean':
            self.slice_selection = 'mean'
        elif cfg['slice_selection'] == 'max':
            self.slice_selection = 'max'
        else:
            raise ValueError('Invalid slice selection method. Choose between "mean" and "max".')

        if cfg['voxel_selection'] == 'mean':
            self.voxel_selection = 'mean'
        elif cfg['voxel_selection'] == 'max':
            self.voxel_selection = 'max'
        else:
            raise ValueError('Invalid voxel selection method. Choose between "mean" and "max".')
        
        
        
        # statistics to track
        self.annotated_pixels = None
        self.annotated_slices = None
        

    def _sum_l1_per_slice(
        self, 
        volume: Tensor
    ) -> Tensor:
        """ Find sum over all slices in each direction

        Parameters
        ----------
        volume : Tensor 
            shape L x W x H with L = W = H  

        Returns
        -------
        out : Tensor
            (N, n) shaped array holding slice sums, where N is
            the dimensionality (3) and n the number of slices in each
            direction (L, W, H). Expects zero padding to ensure same length
            across dimensions.
        """

        assert((volume.shape[0] == volume.shape[1]) and (volume.shape[0] == volume.shape[2]))

        # dimensionality and array of possible axis for volume
        dims    = len(volume.shape)
        indices = torch.arange(dims)
        sums    = torch.zeros((dims, volume.shape[0]))

        # for each direction, sum values in each slice
        for dim in range(dims):
            axis       = tuple(indices[indices != dim])
            slice_sums = volume.sum(axis=axis)
            sums[dim]  = slice_sums
        return sums
        
        
    def _order_slices_by_sum(
        self, 
        slice_sums: Tensor
    ) -> Union[np.array, np.array]:
        """ Order slices by their overall sum
        Note: numpy dependent, because np.unravel_index exists

        Parameters
        ----------
        slice_sums : Tensor
            shape (N, n) with N=3 and n=L=H=W

        Returns
        -------
        axis : 1d array
            axis of slices in descending order w.r.t.
            their slice sum
        slices : 1d array
            slice index of slices in descending order
            w.r.t. their slice sum
        """

        length = slice_sums.shape[0]

        # calculate direction (axis) and indices of slices in 
        # descending order w.r.t. their slice sum
        sorted_slice_indices = torch.argsort(slice_sums.flatten(), descending=True)
        # Note: Unravel_index is not yet implemented in torch, but a requested feature
        #       as of Dec 2020. Maybe add later. 
        axis, slices = np.unravel_index(sorted_slice_indices, slice_sums.shape)
        
        return axis, slices


    def _slice_samples_per_class(
        self, 
        slc: Tensor, 
        inverse_frequencies: Tensor,
        n: int
    ) -> Tensor:
        """ samples seeds for each class in a slice from the
            error map, weighted by inverse class frequencies

        Parameters
        ----------
        slc : Tensor
            slice from error map, shape n_classes x W x H

        inverse_frequencies : Tensor
            inverse class frequencies from ground truth, shape n_classes x 1 x 1 x 1

        n : int
            number of seeds

        Returns
        -------
        n_samples : Tensor
            shape n_classes

        """
        # omit sign for number of misclassifications
        slc_abs = torch.abs(slc)

        # calculate proportions by dividing total number of 
        # misclassifications by class frequency for each class
        # and then normalize to stochastic vector
        total                  = slc_abs.sum(dim=(1,2))
        proportions            = total * inverse_frequencies.flatten()
        proportions_normalized = proportions / proportions.sum()

        # quantize sample proportions to get preliminary number
        # of samples
        n_samples = (proportions_normalized * n).type(torch.int)

        # catch cases where int conversion results in a number of
        # samples that is different from n and correct them.
        # Current strategy: minimal impact by removing and adding
        # to dominant class
        while ( n_samples.sum() != n ):
            # in case of undershoot, add samples to class
            # with highest number of overall samples
            if n_samples.sum() < n:
                n_samples[np.argmax(proportions_normalized)] += 1

            # in case of overshoot, remove samples from class
            # with highest number of overall samples        
            else:
                n_samples[np.argmax(proportions_normalized)] -= 1

        print(n_samples)
        return n_samples


    def _sample_candidate_voxels(
        self, slc: Tensor, 
        ground_truth_slice: Tensor, 
        n_class_samples: Tensor, 
        seed=None
    ) -> Tensor:
        """ individually sample voxels for each class with samples sizes
            potentially varying among them.

        Parameters
        ----------
        slc : Tensor
            slice from error map, shape n_classes x W x H
            
        ground_truth_slice : Tensor
            ground truth slice, shape n_classes x W x H
            
        n_class_samples : Tensor
            number of samples for each class, shape n_classes
            
        seed : int
            If not None (default), set specified seed
            before sampling.

        Returns
        -------
        samples : Tensor
            mask with samples for specified slice and all classes,
            shape n_classes x W x H
        """

        # seed if specified
        if seed is not None:
            torch.manual_seed(seed)

        # init sampler and output tensor
        sampler = torch.utils.data.WeightedRandomSampler
        samples = torch.zeros_like(slc)

        #weights = torch.any(torch.abs(slc).type(torch.uint8), axis=0) * ground_truth_slice # 5 x 145 x 145 , alte Version
        weights = torch.abs(slc).max(dim=0).values * ground_truth_slice # 5 x 145 x 145 , neue Version

        # print(weights.shape, (weights>0).sum(axis=(1,2)))
        weights = weights / (weights>0).sum(axis=(1,2)).reshape(-1,1,1)
        # iterate over classes, sampling for each independently
        for weight, num_samples, i in zip(weights, n_class_samples, range(slc.shape[0])):
            
            # generate uniform weights for false negative voxels
            # weight = (volume > 0) / (volume > 0).sum()
            # upper bound for number of samples to maximum in slice
            max_samples = (weight > 0).sum()
            num_samples = int(min(max_samples, num_samples))
            # catch case where number of samples is zero for a class
            if num_samples > 0:
                # 1D coordinates for samples from weight matrix
                #print(num_samples)
                index_list = list(sampler(weight.flatten(), num_samples=num_samples, replacement=False))
                # 2D coordinates for samples from weight matrix
                index_coords = np.unravel_index(index_list, weight.shape)
                # apply mask via coordinates to samples for class i
                # print(samples.shape)
                for l in range(len(samples)):
                    samples[l][index_coords] = ground_truth_slice[l][index_coords]
                # print(samples.sum())
                # # samples[i][index_coords] = 1   # alte Version, annotiert nur aktuell betrachtete Klasse
                # samples.view(n_class_samples.shape[0], -1)[index_list] = ground_truth_slice.view(n_class_samples.shape[0], -1)[index_list]
                # # samples[:, index_coords] = ground_truth_slice[:, index_coords] # TODO: get this to run instead of the old version
                # print(samples.sum())
                #print(index_coords)

        return samples


    def sample_random_candidate_voxels(
        self, 
        slc: Tensor, 
        ground_truth_slice: Tensor, 
        n_samples, 
        seed=None
    ) -> Tensor:

        # seed if specified
        #if seed is not None:
        #    torch.manual_seed(seed)

        # init sampler and output tensor	
        sampler = torch.utils.data.WeightedRandomSampler
        samples = torch.zeros_like(slc)

        weights = torch.any(torch.abs(slc).type(torch.uint8), axis=0) # 145 x 145
        weights = weights / (weights>0).sum(axis=(0,1))

        num_samples = int(min((weights > 0).sum(), n_samples))

        index_list = list(sampler(weights.flatten(), num_samples=num_samples, replacement=False))
        index_coords = np.unravel_index(index_list, weights.shape)

        samples[:, index_coords] = ground_truth_slice[:, index_coords]

        return samples # has to have shape [5, 145, 145]


    def _slice_add_neighbors(self, class_samples: Tensor, ground_truth_slice: Tensor) -> Tensor:
        """ creates slice with all sampled interaction candidates and their
            neighborhoods for each class

        Parameters
        ----------
        class_samples : Tensor
            sampled seed mask, shape n_classes x W x H 
    
        ground_truth : Tensor
            ground truth slice, shape n_classes x W x H 


        Returns
        -------
        interaction_mask : Tensor
            mask with added neighbors from brushing,
            shape n_classes x W x H
        """
        interaction_mask           = torch.zeros_like(class_samples, dtype=torch.int64)
        vectorized_binary_erosion  = np.vectorize(binary_erosion,  signature='(j,i),(k,k)->(j,i)')
        vectorized_binary_dilation = np.vectorize(binary_dilation, signature='(j,i),(k,k)->(j,i)')


        for size in self.brush_sizes:
            brush             = torch.ones((size,size))
            brushable_samples = class_samples * torch.tensor(vectorized_binary_erosion(ground_truth_slice, structure=brush))
            brushed_samples   = vectorized_binary_dilation(brushable_samples, structure=brush).astype(int)
            interaction_mask  = torch.bitwise_or(interaction_mask, torch.tensor(brushed_samples))
        
        return interaction_mask


    def initial_annotation(
        self, 
        n_samples: int, 
        init: str = 'three_slices', 
        pos_weight: float = 1, 
        seed: int = 42
    ) -> Tensor:        
        """ creates the initial annotations. For each direction (saggital, coronal,
            axial), select the slice with the most foreground labels (3 in total).
            (2) For each slice, sample n_samples many seeding points and
                save their position in an annotation mask.
            (3) Apply the largest quadratic brush (from a given range) to each seed
                for which all affected voxels are foreground and add them to
                the annotation mask as well.
            (4) Mask the ground truth labels with the annotation mask and return.
        
        Parameters
        ----------
        n_samples : int
            number of seed points for each slice. Not needed when 
            init = paper_init
        
        init : str
            'three_slices': Finds three slices (one in each direction) that have
                a high label densety across all classes and then annotates them
                partly.
            'paper_init': the initial annotation matches the annotation used in 
                preliminary paper.
            'per_class': For each class, finds one slice with high label density
                and annotates it partly. The annotations per slice are distributed
                according to #TODO
        
        pos_weight : weight for forground vs background samples per slice. Only used
            if init='per_class'.

        Returns
        -------
        interaction_map : Tensor
            shape n_classes x L x W x H

        """
        interaction_map = torch.zeros_like(self.gt).int()
        if init == 'paper_init':
            # [("sagittal", 72) -> 72, ("coronal", 87) -> 72, ("axial", 72) -> 72]
            
            for orientation in range(3):
                selection = [slice(None)] * 4
                if orientation == 0:
                    selection[orientation + 1]  = [73]
                else:
                    selection[orientation + 1]  = [72]
                interaction_map[selection] = self.gt[selection]

        elif init == 'three_slices':
            n_classes = self.gt.shape[0]
            #t = self.gt

            inverse_size_weights = self.gt.mean((1,2,3)).sum() / self.gt.mean((1,2,3)).reshape((n_classes,1,1,1))
            
            
            t_norm = torch.norm(self.gt * inverse_size_weights, p=1, dim=0)

            # 1.1) calc sum over l1 norms, e.g. for the l1 norms for segmentation predictions
            slice_sums = self._sum_l1_per_slice(t_norm)

            # 1.2) order slices in descending order by their sum
            axis, indices = self._order_slices_by_sum(slice_sums)

            # save data location for later sub - sampling
            data_location = []

            # select one slice for each direction in volume.
            for orientation in range(3):
                #Use the ordered slice list to find best slice for each direction
                selection = [slice(None)] * 4
                index     = indices[np.argmax(axis == orientation)]
                selection[orientation+1] = index
                # select slice from misclassifications and ground truth
                t_selection = self.gt[selection]

                # samples voxels and add their neighborhood to 2D mask
                n_class_samples = self._slice_samples_per_class(t_selection, inverse_size_weights, n_samples // 3) 
                #print(n_class_samples.sum())
                #print(t_selection.shape, n_class_samples)

                class_samples = self._sample_candidate_voxels(t_selection, t_selection, n_class_samples=n_class_samples, seed=seed)
                brushed_mask  = self._slice_add_neighbors(class_samples, t_selection)
                # make interaction map with same shape as model input
                interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
                # interaction_map[selection] = ((interaction_map[selection].sum(0) * t_selection) > 0) * 1
                data_location.append((orientation, index))
        
        elif init == 'per_class':
            n_classes = self.gt.shape[0]
            data_location = []
            inverse_size_weights = self.gt.mean((1,2,3)).sum() / self.gt.mean((1,2,3)).reshape((n_classes,1,1,1))
            
            for c in range(n_classes):
                cweight = torch.eye(n_classes)[c].view(n_classes, 1, 1, 1)
                t_norm = torch.norm(self.gt * cweight, p=1, dim=0)
                slice_sums = self._sum_l1_per_slice(t_norm)
                axis, indices = self._order_slices_by_sum(slice_sums)
                
                selection = [slice(None)] * 4
                selection[axis[0] + 1] = indices[0]
                
                t_selection = self.gt[selection]
                
                slice_sample_weights = inverse_size_weights * cweight * n_classes / (n_classes+1) * pos_weight + \
                                    inverse_size_weights * (1-cweight) / ( (n_classes+1) * (n_classes-1) )
                
                n_class_samples = self._slice_samples_per_class(t_selection, slice_sample_weights, n_samples)
                #print(n_class_samples, n_samples)
                class_samples = self._sample_candidate_voxels(t_selection, t_selection, n_class_samples=n_class_samples, seed=seed)

                brushed_mask  = self._slice_add_neighbors(class_samples, t_selection)
                interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
                data_location.append((axis[0], indices[0]))

        return interaction_map.float()
        
         
    def refinement_annotation(
        self,
        prediction: Tensor, 
        annotation_mask: Tensor, 
        uncertainty_map: Tensor,
        n_samples: int, 
        mode: int = 'single_slice', 
        pos_weight: float = 1, 
        seed: int = 42
    ) -> Tensor:
        """ Finds the slice with the worst prediction across all three axis and 
            annotates parts of it. The annotation happens in multiple steps:
            (1) mask all voxels that are already annotated with annotation_mask
            (2) Sample n_samples many seeding points and save their position in
                an annotation mask
            (3) Apply the largest quadratic brush (from a given range) to each seed
                for which all affected voxels are foreground and add them to
                the annotation mask as well.
            (4) Mask the ground truth labels with the annotation mask and return

        Parameters
        ----------
        prediction : Tensor
            predictions of segmentation model with
            shape n_classes x L x W x H

        annotation_mask : Tensor
            current annotation, shape n_classes x L x W x H

        n_samples : int
            number of samples per slice before brushing

        Returns
        -------
        interaction_map : Tensor
            new annotations, shape n_classes x L x W x H

        """
        n_classes = prediction.shape[0]

        # calculate inverse class frequencies
        inverse_size_weights = self.gt.mean((1,2,3)).sum() / self.gt.mean((1,2,3)).reshape((n_classes,1,1,1))

        # calculate mask for available voxels
        #available_voxels = 1 - annotation_mask.float() # alte Version
        available_voxels = 1 - torch.any(annotation_mask, dim=0, keepdim=True) * 1

        # calculate difference between truth and prediction, i.e. misclassified voxels
        if uncertainty_map != None:
            diff = uncertainty_map * available_voxels
        else:
            diff = torch.abs(self.gt - prediction.float()) * available_voxels
        # print("weights:",inverse_size_weights.flatten())
        
        
        if mode == 'single_slice':
            # norm over classes weighted by inverse class frequency - importance weight for sampling
            diff_norm = torch.norm(diff  * inverse_size_weights, p=1, dim=0)

            # 1.1) calc sum over l1 norms, e.g. for the l1 norms for segmentation predictions
            slice_sums = self._sum_l1_per_slice(diff_norm)

            # 1.2) order slices in descending order by their sum
            axis, indices = self._order_slices_by_sum(slice_sums)

            # 2.0) select slice with highest importance weight over all axes
            random_selection = np.random.randint(0,6)
            ax  = axis[0]
            slc = indices[0]
            data_location = (ax, slc)
            selection = [slice(None)] + [slice(None)] * 3
            selection[ax + 1] = slc

            # 2.1) calculate number of samples for each class from a raw difference slice
            diff_selection  = diff[selection]
            t_selection     = self.gt[selection]
            n_class_samples = self._slice_samples_per_class(diff_selection, inverse_size_weights, n_samples)
            #print(n_class_samples.sum())

            # 2.2) for each class, sample from false negatives as often as specified in n_class_samples
            class_samples = self._sample_candidate_voxels(diff_selection, t_selection, n_class_samples=n_class_samples, seed=seed)

            # 2.3) brush all samples with maximum brush from list of brushes
            brushed_mask = self._slice_add_neighbors(class_samples, t_selection)

            # 2.4) create interaction map to return
            interaction_map = torch.zeros_like(self.gt, dtype=torch.int64)
            interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
            # interaction_map[selection] = ((interaction_map[selection].sum(0) * t_selection) > 0) * 1
                
        elif mode == 'per_class':
            data_location = []
            interaction_map = torch.zeros_like(self.gt, dtype=torch.int64)
            for c in range(n_classes):
                cweight = torch.eye(n_classes)[c].view(n_classes, 1, 1, 1)
                diff_norm = torch.norm(diff * cweight, p=1, dim=0) # 145, 145, 145, binär

                # 1.1) calc sum over l1 norms, e.g. for the l1 norms for segmentation predictions
                slice_sums = self._sum_l1_per_slice(diff_norm)  # 3, 145

                # 1.2) order slices in descending order by their sum
                axis, indices = self._order_slices_by_sum(slice_sums)

                # 2.0) select slice with highest importance weight over all axes
                random_selection = np.random.randint(0,6)
                ax  = axis[0]
                slc = indices[0]
                
                selection = [slice(None)] + [slice(None)] * 3
                selection[ax + 1] = slc

                # 2.1) calculate number of samples for each class from a raw difference slice
                diff_selection = diff[selection]
                t_selection    = self.gt[selection]
                
                slice_sample_weights = inverse_size_weights * cweight * n_classes / (n_classes+1) * pos_weight + \
                    inverse_size_weights * (1-cweight) / ( (n_classes+1) * (n_classes-1) )
                
                n_class_samples = self._slice_samples_per_class(t_selection, slice_sample_weights, n_samples)
                #print(n_class_samples, n_samples)
                
                # 2.2) for each class, sample from false negatives as often as specified in n_class_samples
                class_samples = self._sample_candidate_voxels(diff_selection, t_selection, n_class_samples=n_class_samples, seed=seed)

                # 2.3) brush all samples with maximum brush from list of brushes
                brushed_mask = self._slice_add_neighbors(class_samples, t_selection)

                # 2.4) create interaction map to return
                interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
                data_location.append((axis[0], indices[0]))
                
        return interaction_map.float() # , selection

    
    def random_refinement_annotation(
        self, 
        prediction: Tensor, 
        annotation_mask: Tensor,
        brain_mask: Tensor, 
        n_samples: int, 
        mode: int = 'single_slice', 
        pos_weight: float = 1, 
        seed: int = 42
    ) -> Tensor:
        """ Finds the slice with the highest uncertainty across all three axis and 
            annotates parts of it. The annotation happens in multiple steps:
            (1) mask all voxels that are already annotated with annotation_mask
            (2) Sample n_samples many seeding points and save their position in
                an annotation mask
            (3) Apply the largest quadratic brush (from a given range) to each seed
                for which all affected voxels are foreground and add them to
                the annotation mask as well.
            (4) Mask the ground truth labels with the annotation mask and return

        Parameters
        ----------
        prediction : Tensor
            predictions of segmentation model with
            shape n_classes x L x W x H

        annotation_mask : Tensor
            current annotation, shape n_classes x L x W x H

        n_samples : int
            number of samples per slice before brushing

        Returns
        -------
        interaction_map : Tensor
            new annotations, shape n_classes x L x W x H

        """
        n_classes = prediction.shape[0]

        # calculate inverse class frequencies
        inverse_size_weights = self.gt.mean((1,2,3)).sum() / self.gt.mean((1,2,3)).reshape((n_classes,1,1,1))

        # calculate mask for available voxels
        available_voxels = 1 - torch.any(annotation_mask, dim=0, keepdim=True) * 1
    	
        annotated_voxels = torch.any(annotation_mask, axis=0)
        brain_not_annoated_mask = brain_mask & ~annotated_voxels
        x = torch.zeros((5,145,145,145))
        x[:, brain_not_annoated_mask] = 1     # 5, 145, 145, 145
        random_mask = torch.zeros((145,145,145))
        random_mask[brain_not_annoated_mask] = 1   # 145, 145, 145

        if mode == 'single_slice':
            #np.random.seed(seed)
            random_axis = np.random.randint(0,3)
            match random_axis:
                case 0:
                    slice_sums = torch.sum(random_mask, axis=(1,2))
                case 1:
                    slice_sums = torch.sum(random_mask, axis=(0,2))
                case 2:
                    slice_sums = torch.sum(random_mask, axis=(0,1))

            valid_slice_indices = torch.where(slice_sums >= n_samples)[0]
            random_slice_index = np.random.choice(valid_slice_indices)     

            ax = random_axis
            slc = random_slice_index
            #print(ax, slc)
            data_location = (ax, slc)
            selection = [slice(None)] + [slice(None)] * 3
            selection[ax + 1] = slc

            random_selection = x[selection]
            t_selection = self.gt[selection]

            samples = self.sample_random_candidate_voxels(random_selection, t_selection, n_samples)
            brushed_mask = self._slice_add_neighbors(samples, t_selection)

            interaction_map = torch.zeros_like(self.gt, dtype=torch.int64)
            interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)

        elif mode == 'per_class':
            data_location = []
            interaction_map = torch.zeros_like(self.gt, dtype=torch.int64)
            for c in range(n_classes):
                slice_sums = self._sum_l1_per_slice(random_mask)
                random_axis = np.random.randint(0,3)
                match random_axis:
                    case 0:
                        slice_sums = slice_sums[0]
                    case 1:
                        slice_sums = slice_sums[1]
                    case 2:
                        slice_sums = slice_sums[2]
                
                valid_slice_indices = torch.argwhere(slice_sums > 0).flatten()    # NOTE: oder größer gleich n_samples?
                random_slice_index = np.random.choice(valid_slice_indices)

                ax = random_axis
                slc = random_slice_index

                selection = [slice(None)] + [slice(None)] * 3
                selection[ax + 1] = slc
                random_selection = x[selection]
                t_selection = self.gt[selection]

                samples = self.sample_random_candidate_voxels(random_selection, t_selection, n_samples, seed=seed)
                brushed_mask = self._slice_add_neighbors(samples, t_selection)
                interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
                
        return interaction_map.float() # , selection


    # def novelty_refinement_annotation(
    #     self,
    #     annotation_mask: Tensor, 
    #     novelty_map: Tensor,
    #     n_samples: int, 
    #     mode: int = 'per_class', 
    #     pos_weight: float = 1, 
    #     seed: int = 42
    # ):
    
    #     n_classes = annotation_mask.shape[0]

    #     # calculate inverse class frequencies
    #     inverse_size_weights = self.gt.mean((1,2,3)).sum() / self.gt.mean((1,2,3)).reshape((n_classes,1,1,1))

    #     # calculate mask for available voxels
    #     #available_voxels = 1 - annotation_mask.float() # alte Version
    #     available_voxels = 1 - torch.any(annotation_mask, dim=0, keepdim=True) * 1

    #     # mask novelty score for all voxels that are already annotated
    #     diff = novelty_map * available_oxels

    #     if mode == 'per_class':
    #         data_location = []
    #         interaction_map = torch.zeros_like(self.gt, dtype=torch.int64)
    #         for c in range(n_classes):
    #             cweight = torch.eye(n_classes)[c].view(n_classes, 1, 1, 1)
    #             diff_norm = torch.norm(diff * cweight, p=1, dim=0) # 145, 145, 145, binär

    #             # 1.1) calc sum over l1 norms, e.g. for the l1 norms for segmentation predictions
    #             slice_sums = self._sum_l1_per_slice(diff_norm)  # 3, 145

    #             # 1.2) order slices in descending order by their sum
    #             axis, indices = self._order_slices_by_sum(slice_sums)

    #             # 2.0) select slice with highest importance weight over all axes
    #             random_selection = np.random.randint(0,6)
    #             ax  = axis[0]
    #             slc = indices[0]
                
    #             selection = [slice(None)] + [slice(None)] * 3
    #             selection[ax + 1] = slc

    #             # 2.1) calculate number of samples for each class from a raw difference slice
    #             diff_selection  = diff[selection]
    #             t_selection     = self.gt[selection]
                
    #             slice_sample_weights = inverse_size_weights * cweight * n_classes / (n_classes+1) * pos_weight + \
    #                                 inverse_size_weights * (1-cweight) / ( (n_classes+1) * (n_classes-1) )
                
    #             n_class_samples = self._slice_samples_per_class(t_selection, slice_sample_weights, n_samples)
    #             #print(n_class_samples, n_samples)
                
    #             # 2.2) for each class, sample from high novelty areas as often as specified in n_class_samples
    #             class_samples = self._sample_candidate_voxels(diff_selection, t_selection, n_class_samples=n_class_samples, seed=seed)

    #             # 2.3) brush all samples with maximum brush from list of brushes (deprecated. Brush size is usually 1)
    #             brushed_mask = self._slice_add_neighbors(class_samples, t_selection)

    #             # 2.4) create interaction map to return
    #             interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
    #             data_location.append((axis[0], indices[0]))
                
    #     return interaction_map.float() # , selection



        # """ Finds the slice with the worst prediction across all three axis and 
        #     annotates parts of it. The annotation happens in multiple steps:
        #     (1) mask all voxels that are already annotated with annotation_mask
        #     (2) Sample n_samples many seeding points and save their position in
        #         an annotation mask
        #     (3) Apply the largest quadratic brush (from a given range) to each seed
        #         for which all affected voxels are foreground and add them to
        #         the annotation mask as well.
        #     (4) Mask the ground truth labels with the annotation mask and return

        # Parameters
        # ----------
        # prediction : Tensor
        #     predictions of segmentation model with
        #     shape n_classes x L x W x H

        # annotation_mask : Tensor
        #     current annotation, shape n_classes x L x W x H

        # n_samples : int
        #     number of samples per slice before brushing

        # Returns
        # -------
        # interaction_map : Tensor
        #     new annotations, shape n_classes x L x W x H

        # """
        # # n_classes = prediction.shape[0]

        # # calculate inverse class frequencies
        # # inverse_size_weights = self.gt.mean((1,2,3)).sum() / self.gt.mean((1,2,3)).reshape((n_classes,1,1,1))

        # # calculate mask for available voxels
        # #available_voxels = 1 - annotation_mask.float() # alte Version
        # available_voxels = 1 - torch.any(annotation_mask, dim=0, keepdim=True) * 1

        # # calculate novelty scores within the available voxels
        # diff = novelty_map * available_voxels

        # # 1.1) calc slice wise scores as sum over all pixels within the slice
        # slice_sums = self._sum_l1_per_slice(diff)

        # # 1.2) order slices in descending order by their sum
        # axis, indices = self._order_slices_by_sum(slice_sums)

        # # 2.0) select slice with highest importance weight over all axes
        # random_selection = np.random.randint(0,6)
        # ax  = axis[0]
        # slc = indices[0]
        # data_location = (ax, slc)
        # selection = [slice(None)] + [slice(None)] * 3
        # selection[ax + 1] = slc

        # # 2.1) calculate number of samples for each class from a raw difference slice
        # diff_selection  = diff[selection]
        # t_selection     = self.gt[selection]
        # # n_class_samples = self._slice_samples_per_class(diff_selection, inverse_size_weights, n_samples)
        # #print(n_class_samples.sum())

        # # 2.2) for each class, sample from false negatives as often as specified in n_class_samples
        # class_samples = self._sample_candidate_voxels(diff_selection, t_selection, n_class_samples=n_class_samples, seed=seed)

        # # 2.3) brush all samples with maximum brush from list of brushes
        # brushed_mask = self._slice_add_neighbors(class_samples, t_selection)

        # # 2.4) create interaction map to return
        # interaction_map = torch.zeros_like(self.gt, dtype=torch.int64)
        # interaction_map[selection] = torch.bitwise_or(interaction_map[selection], brushed_mask)
        # # interaction_map[selection] = ((interaction_map[selection].sum(0) * t_selection) > 0) * 1
            

In [21]:
class EvalDataset(Dataset):
    
    def __init__(
        self,
        subject_id: str,
        cfg, 
        modality='reconstruction', 
        mode='train', 
        to_gpu=True, 
        init='three_slices', 
    ):
        super().__init__()
        self.cfg          = OmegaConf.to_container(cfg)
        self.axis         = cfg.data.axis
        self.labels       = cfg.data.labels
        self.device       = cfg.rank
        self.modality     = modality
        self.mode         = mode
        self.to_gpu       = to_gpu
        self.init         = init

        self.data_path = os.path.join(
            cfg.data.data_dir, 
            str(subject_id), 
            "Diffusion", 
            "data.nii.gz"
        )
        self.data_in = torch.tensor(nib.load(self.data_path).get_fdata()).float()

        self.mask_path = os.path.join(
            cfg.data.data_dir, 
            str(subject_id), 
            "Diffusion", 
            "nodif_brain_mask.nii.gz"
        )
        self.brain_mask = torch.tensor(
            nib.load(self.mask_path).get_fdata(), dtype=torch.bool)
        
        self.tract_path = os.path.join(
            cfg.data.data_dir, 
            str(subject_id), 
            "tracts_masks"
        )
        self.get_tract_masks(self.labels)

        if self.axis == 'coronal':
            self.data_in = self.data_in.permute(1,3,2,0).rot90(2, dims=[1,2])[14:159]
            self.brain_mask = self.brain_mask.permute(1,2,0).rot90(2, dims=[1,2])[14:159]
            self.label = self.label.permute(0,2,3,1).rot90(2, dims=[2,3])[:,14:159]
        else:
            raise NotImplementedError("Only coronal axis is supported at the moment")

        # if cfg['log']:
        #    wandb.config.update({'labels': cfg['labels']})
            
        self.user = UserModel(self.label, cfg)
            
        # [classes, B, H, W]
        self.annotations = None

        # [B, 1, H, W]
        self.weight = None

        self.pos_weight = (
            len(self.labels)*self.brain_mask.sum() - \
            self.label.sum((1,2,3))[None, :, None, None]
        ) / self.label.sum((1,2,3))[None, :, None, None]
        
        if self.to_gpu:
            self.data_in    = self.data_in.to(self.device)
            self.label      = self.label.to(self.device)
            self.brain_mask = self.brain_mask.to(self.device) 
            self.pos_weight = self.pos_weight.to(self.device) 


    def get_tract_masks(
        self,
        labels: List[str]
    ) -> Tensor:
        self.label = None
        for tract in self.labels:
            
            if f'{tract}.nii.gz' in os.listdir(self.tract_path):
                tract_mask = torch.tensor(
                    nib.load(os.path.join(self.tract_path, f'{tract}.nii.gz')).get_fdata()
                ).long()
                if self.label is None:
                    self.label = tract_mask.unsqueeze(0)
                else:
                    self.label = torch.cat([
                        self.label, 
                        tract_mask.unsqueeze(0)
                    ], dim=0)

            # class other is handled below
            elif tract == 'Other':
                continue
            else:
                # left and right are different classes but the raw data makes
                # even more distictions we don't care for
                if 'left' in tract or 'right' in tract:
                    tract = tract.split('.')[0]
                    tract_parts = tract.split('_')
                    tract, side = tract_parts[0], tract_parts[-1]
                    # tract, side = tract.split('_')
                else:
                    side = ''
                tract_files = [
                    f for f in os.listdir(self.tract_path)
                    if tract in f and side in f
                ]

                tract_mask = None
                for f in tract_files:
                    tract_mask_tmp = torch.tensor(
                        nib.load(os.path.join(self.tract_path, f)).get_fdata()
                    ).long()
                    if tract_mask is None:
                        tract_mask = tract_mask_tmp
                    else:
                        tract_mask = torch.bitwise_or(
                            tract_mask, tract_mask_tmp
                        )

                if self.label is None:
                    self.label = tract_mask.unsqueeze(0)
                else:
                    self.label = torch.cat([
                        self.label, 
                        tract_mask.unsqueeze(0)
                    ], dim=0)


        other = self.brain_mask * ~torch.any(self.label, dim=0)
        self.label = torch.cat([
            other.unsqueeze(0),
            self.label, 
        ], dim=0)
           
        
    def set_mode(self, mode) -> None:
        self.mode = mode


    def set_modality(self, modality) -> None:
        self.modality = modality     
        
        
    def initial_annotation(
        self, 
        seed=42
    ) -> Tensor:
        return self.user.initial_annotation(
            #self.label.detach().cpu(),
            self.cfg["init_voxels"],
            init=self.init, 
            seed=seed
        )
    

    def random_refinement_annotation(
        self,
        prediction, 
        seed=42
    ) -> Tensor:
        
        if self.init == 'per_class':
            mode = 'per_class'
            
        elif self.init == 'three_slices':
            mode = 'single_slice'

        return self.user.random_refinement_annotation(
            prediction, 
            self.annotations.detach().cpu(),
            self.brain_mask.detach().cpu(),
            self.cfg["refinement_voxels"],
            mode=mode,
            seed=seed
        )


    def refinement_annotation(
        self, 
        prediction, 
        uncertainty_map=None, 
        random=False, 
        seed=42
    ) -> Tensor:
        
        if self.init == 'per_class':
            mode = 'per_class'
            
        elif self.init == 'three_slices':
            mode = 'single_slice'
        
        elif random:
            return self.user.random_refinement_annotation(
                prediction, 
                self.annotations.detach().cpu(),
                self.brain_mask.detach().cpu(),
                self.cfg["refinement_voxels"],
                mode=mode,
                seed=seed
            )

        else:
            return self.user.refinement_annotation(
                prediction,
                #self.label.detach().cpu(),
                self.annotations.detach().cpu(),
                uncertainty_map,
                self.cfg["refinement_voxels"],
                mode=mode,
                seed=seed
            )


    def update_annotation(self, annotations) -> None:
        assert(annotations.data.type() == 'torch.FloatTensor')

        if self.to_gpu:
            annotations = annotations.to(self.device)

        if self.annotations is None:
            self.annotations = annotations
        else:
            self.annotations += annotations
            self.annotations  = torch.clamp(self.annotations, 0, 1)
            
        self.weight = (self.annotations.sum(0) > 0).unsqueeze(1).float()
        self.pos_weight  = (self.weight.sum() - self.annotations.sum((1,2,3))[None, :, None, None])
#             self.pos_weight  = (1 - self.annotations.sum((1,2,3))[None, :, None, None])
        self.pos_weight /= self.annotations.sum((1,2,3))[None, :, None, None]

    
    def clear_annotation(self) -> None:
        self.annotations = None
        

    def __len__(self) -> int:
        return self.data_in.shape[0]
        

    def __getitem__(self, index) -> dict:

        input_ = self.data_in[index]

        if self.modality == 'reconstruction':
            target = self.data_in[index].detach().clone()
            weight = 1.
            
        elif self.modality == 'segmentation':
        
            if self.mode == 'train':
                target = self.annotations[:, index].detach()
            elif self.mode == 'validate':
                target = self.label[:, index].detach()
    

            weight = self.weight[index]

        mask = self.brain_mask[index]
        
        return {
            'input':  input_,
            'target': target,
            'weight': weight, # may needs unsqueeze(0) in validate
            'mask':   mask
        }

In [22]:
# cfg = {
#     # CONFIG
#     'name': 'location-unsupervised',
#     'project': 'IDVR-localization_pretrain',
#     'log': False,
#     'rank': 0,
#     'axis': 'coronal',
#     # 'labels': [
#     #     "Other", 
#     #     "CG", 
#     #     "CST", 
#     #     "FX", 
#     #     "CC"
#     # ],
#     'labels': [
#         "Other", 
#         "IFO_left", 
#         "IFO_right", 
#         "ILF_left",
#         "ILF_right", 
#         "SLF_left", 
#         "SLF_right"
#     ],
#     'slice_selection': 'mean',
#     'voxel_selection': 'mean',
    
#     # DATA
#     'data_dir': '../../../../../data/hcp',
#     'data_path': '../../../data/784565/Diffusion/data.nii',
#     'active_mask_path': '../../../data/784565/Diffusion/nodif_brain_mask.nii.gz',
    
#     # SELF SUPERVISED PRE-TRAINING
#     's_n_epochs': 20,
#     's_batch_size': 16, # default: 8
#     's_lr': 5e-4, #1e-4, 1e-5        
    
#     # TRAINING WITH WEAK SUPERVISION
#     'p_n_epochs': 100,
#     'w_n_epochs': 10,
#     'w_batch_size': 2,
#     'w_lr': 5e-4,    #5e-5 
#     'w_eval_freq': 100,
    
#     # RANDOM FOREST
#     'min_samples_leaf': 8,
    
#     # USER MODEL
#     'init_voxels': 200,
#     'refinement_voxels': 200,
#     'num_interactions': 10,
# }

cfg = OmegaConf.load('../configs/eval.yaml')

dataset = EvalDataset(
    subject_id=729254, 
    cfg=cfg, 
    modality='reconstruction',
    to_gpu=False
)

In [23]:
dataset.user = UserModel(dataset.label, cfg)
dataset.init = 'per_class'
# currently, there are no annotations. We can also enforce this with clear_annotations() at any point
dataset.clear_annotation()
# get initial annotations
annot = dataset.initial_annotation(seed=42)
# and update the dataset
dataset.update_annotation(annot)
print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")

tensor([97,  0,  2,  0,  1], dtype=torch.int32)
tensor([ 1, 97,  1,  0,  1], dtype=torch.int32)
tensor([ 0,  0, 98,  2,  0], dtype=torch.int32)
tensor([  0,   0,   0, 100,   0], dtype=torch.int32)
tensor([ 1,  4,  3,  0, 92], dtype=torch.int32)
number of annotations: 663.0


In [24]:
# get model
model, state_dict = get_model(
    cfg=cfg,
    return_state_dict=True
)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [25]:
# define the layer you want the features from. This is usually the encoder output.
f_layer = 'encoder'
# Init the feature extractor. Have a look at PyTorchs Hook functionality.
extractor = FeatureExtractor(model, layers=[f_layer])
# Cache all features for a dataset and reformat/move to numpy for random forest stuff
hooked_results  = extractor(dataset)
features = hooked_results[f_layer]
features = features.permute(0,2,3,1).numpy()

In [26]:
scores, prediction, uncertainty_maps, uncertainty_per_class_maps = evaluate_RF(
    dataset, 
    features, 
    cfg,
    uncertainty_measures=['entropy-truth']
)

In [27]:
scores

{'Other_precision': array(0.85041994, dtype=float32),
 'Other_recall': array(0.30600485),
 'Other_f1': array(0.45006883),
 'CG_precision': array(0.24978456, dtype=float32),
 'CG_recall': array(0.65688817),
 'CG_f1': array(0.36194695),
 'CST_precision': array(0.2541128, dtype=float32),
 'CST_recall': array(0.52359891),
 'CST_f1': array(0.34217429),
 'FX_precision': array(0.04507279, dtype=float32),
 'FX_recall': array(0.71396894),
 'FX_f1': array(0.08480468),
 'CC_precision': array(0.40540728, dtype=float32),
 'CC_recall': array(0.68015717),
 'CC_f1': array(0.50801798),
 'Avg_prec_tracts': array(0.23859435, dtype=float32),
 'Avg_recall_tracts': array(0.6436533),
 'Avg_f1_tracts': array(0.32423597)}

In [29]:
scores, prediction, uncertainty_maps, uncertainty_per_class_maps = evaluate_RF(
    dataset, 
    features, 
    cfg,
    uncertainty_measures=['entropy']
)

In [30]:
scores

{'Other_precision': array(0.85041994, dtype=float32),
 'Other_recall': array(0.30600485),
 'Other_f1': array(0.45006883),
 'CG_precision': array(0.24978456, dtype=float32),
 'CG_recall': array(0.65688817),
 'CG_f1': array(0.36194695),
 'CST_precision': array(0.2541128, dtype=float32),
 'CST_recall': array(0.52359891),
 'CST_f1': array(0.34217429),
 'FX_precision': array(0.04507279, dtype=float32),
 'FX_recall': array(0.71396894),
 'FX_f1': array(0.08480468),
 'CC_precision': array(0.40540728, dtype=float32),
 'CC_recall': array(0.68015717),
 'CC_f1': array(0.50801798),
 'Avg_prec_tracts': array(0.23859435, dtype=float32),
 'Avg_recall_tracts': array(0.6436533),
 'Avg_f1_tracts': array(0.32423597)}

In [ ]:
def print_results(n_annots: list, f1_scores: list):
    print(f'Iteration | # Annotations | F1 Score')
    print(f'----------|---------------|---------')
    for i, (n, f1) in enumerate(zip(n_annots, f1_scores)):
        if i in [0, 1, 2, 3, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54]:
            print(f'{i+1:>9} | {int(n):>13} | {f1:.4f}')


def re_init_dataset():
    dataset.clear_annotation()
    annot = dataset.initial_annotation(seed=42)
    dataset.update_annotation(annot)


def train(
    method: str, 
    n_epochs: int, 
    measures: List[str]
):
    re_init_dataset()
    print(f'Selection using {method}')
    #print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")
    scores, rf_prediction, unc, unc_pc = evaluate_RF(dataset, features, cfg, measures)
    # scores, rf_prediction, unc, unc_pc = evaluate_RF_with_uncertainty(dataset, features, cfg, measures)
    #scores, rf_prediction = evaluate_RF(dataset, features, cfg)
    print(f"Number of initial annotations: {dataset.annotations.detach().cpu().sum()}")
    print(f"Average F1 score for RF after initial user interaction:    {scores['Avg_f1_tracts'].item():.4f}")
    print()
    n_annots =       []
    annots =         []
    f1_scores =      []
    rf_predictions = []
    uncs_pc =        []
    uncs =           []


    for i in tqdm(range(n_epochs), desc='User interaction', unit='iteration'):
        #print(f"Iteration {i+1}")
        if method == 'random':
            annot = dataset.refinement_annotation(prediction=rf_prediction, random=True, seed=42)
        elif method == 'ground-truth':
            annot = dataset.refinement_annotation(prediction=rf_prediction, seed=42)
        else:
            annot = dataset.refinement_annotation(prediction=rf_prediction, uncertainty_map=unc_pc[method], seed=42)
            #annot = dataset.uncertainty_refinement_annotation(prediction=rf_prediction, uncertainty_map=unc_pc[method], seed=42)

        dataset.update_annotation(annot)
        
        annots.append(dataset.annotations.detach().cpu())
        n_annots.append(dataset.annotations.detach().cpu().sum().item())
        #print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")
        scores, rf_prediction, unc, unc_pc = evaluate_RF(dataset, features, cfg, measures)
        #scores, rf_prediction, unc, unc_pc = evaluate_RF_with_uncertainty(dataset, features, cfg, measures)
        #scores, rf_prediction = evaluate_RF(dataset, features, cfg)
        rf_predictions.append(rf_prediction)
        #uncs_pc.append(unc_pc)
        #uncs.append(unc)
        f1_scores.append(scores['Avg_f1_tracts'].item())
        #print(f"Average F1 score for RF after additional user interaction: {scores['Avg_f1_tracts'].item():.4f}")
    
    print_results(n_annots, f1_scores)
    return n_annots, annots, f1_scores, rf_predictions, uncs_pc, uncs